In [1]:
import datetime
Startdate=datetime.datetime.now().date()-datetime.timedelta(30)
finishDate=datetime.datetime.now().date()-datetime.timedelta(1)

In [2]:
import pyodbc
import pandas as pd
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = '192.168.101.219' 
database = 'DataforAnalysis' 
username = 'DATALYZER' 
password = 'NMB54321'  
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# select 26 rows from SQL table to insert in dataframe.
# Query to get the maximum date
max_date_query = """
    SELECT DATEADD(day, 1, MAX([Date])) as MaxDate
    FROM [DataforAnalysis].[dbo].[Store_SPC];
"""
max_date_df = pd.read_sql(max_date_query, cnxn)

# Printing the maximum date with one day added
max_date_plus_1 = max_date_df['MaxDate'][0]
print(f"Maximum Date + 1 day: {max_date_plus_1}")

query = f"""
        DECLARE @DynamicSQL AS NVARCHAR(max),
 @Start NVARCHAR(30) = '{max_date_plus_1}',
 @End NVARCHAR(30) = '{finishDate}',
 @Parameter NVARCHAR(30) = 'Set_Dim_B';

-- CTE แรก
WITH BarcodeThan100 AS (
    SELECT [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Date] AS [Date],
           COUNT([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Barcode]) AS [countBarcode]
    FROM [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester]
    INNER JOIN [TransportData].[dbo].[Master_matchings]
        ON [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].Model = [TransportData].[dbo].[Master_matchings].Model
    WHERE [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Date] BETWEEN @Start AND @End

        AND [TransportData].[dbo].[Master_matchings].[createdAt] = (
            SELECT MAX([TransportData].[dbo].[Master_matchings].[createdAt])
            FROM [TransportData].[dbo].[Master_matchings]
        )
    GROUP BY [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Date]
),
XT AS (
    SELECT [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Date] AS [Date],
           CASE WHEN DATEDIFF(SECOND, 0, CAST((LAG([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) OVER (ORDER BY [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] DESC) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) AS TIME)) < 120
                THEN DATEDIFF(SECOND, 0, CAST((LAG([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) OVER (ORDER BY [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] DESC) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) AS TIME))
                WHEN DATEDIFF(SECOND, 0, CAST((LAG([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) OVER (ORDER BY [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] DESC) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) AS TIME)) >= 120
                THEN 0
           END AS [C/T in sec],
           CASE WHEN DATEDIFF(SECOND, 0, CAST((LAG([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) OVER (ORDER BY [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] DESC) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) AS TIME)) >= 120
                THEN DATEDIFF(SECOND, 0, CAST((LAG([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) OVER (ORDER BY [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] DESC) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) AS TIME))
                WHEN DATEDIFF(SECOND, 0, CAST((LAG([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) OVER (ORDER BY [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] DESC) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) AS TIME)) < 120
                THEN 0
           END AS [D/T],
           [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].Set_Dim_B AS [Parameter],
           [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Model] AS [model],
           [TransportData].[dbo].[Master_matchings].LSL AS [LSL],
           [TransportData].[dbo].[Master_matchings].CL AS [CL],
           [TransportData].[dbo].[Master_matchings].USL AS [USL],
           [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Machine_no] AS [MC],
           [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Barcode],
           [Line]
    FROM BarcodeThan100
    LEFT JOIN [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester]
        ON BarcodeThan100.Date = [Dynamic_Parallelism_Tester].Date
    INNER JOIN [TransportData].[dbo].[Master_matchings]
        ON [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].Model = [TransportData].[dbo].[Master_matchings].Model
    WHERE [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Date] BETWEEN @Start AND @End
        AND [TransportData].[dbo].[Master_matchings].[Parameter] = @Parameter
        AND [TransportData].[dbo].[Master_matchings].[createdAt] = (
            SELECT MAX([TransportData].[dbo].[Master_matchings].[createdAt])
            FROM [TransportData].[dbo].[Master_matchings]
        )
        AND countBarcode > 100
        AND Operator = 'Operator'
)
, XO AS (
    SELECT [Date] AS [Date],
           CAST(CONVERT(FLOAT, COUNT(CASE WHEN [Parameter] >= [LSL] AND [Parameter] <= [USL] THEN 1 ELSE NULL END)) /
                CONVERT(FLOAT, COUNT(CASE WHEN [Parameter] >= [LSL] AND [Parameter] <= [USL] THEN 1 ELSE NULL END) +
                COUNT(CASE WHEN [Parameter] < [LSL] OR [Parameter] > [USL] THEN 1 ELSE NULL END)) * 100 AS DECIMAL(10, 2)) AS [%Yield],
           CAST(CONVERT(FLOAT, SUM([C/T in sec])) / COUNT([C/T in sec]) AS DECIMAL(10, 2)) AS [Cycle_time (sec)],
           CAST(CONVERT(FLOAT, SUM([D/T])) / 60 AS DECIMAL(10, 0)) AS [Down time (min)],
           CAST(AVG([Parameter]) AS DECIMAL(10, 3)) AS [AVG],
           CAST(STDEV([Parameter]) AS DECIMAL(10, 4)) AS [STD],
           CASE WHEN (CAST(CONVERT(FLOAT, (([USL] - AVG([Parameter])) / (3 * STDEV([Parameter])))) AS DECIMAL(10, 2)) <
                     CAST(CONVERT(FLOAT, ((AVG([Parameter]) - [LSL]) / (3 * STDEV([Parameter])))) AS DECIMAL(10, 2))) OR
                    ((CAST(CONVERT(FLOAT, (([USL] - AVG([Parameter])) / (3 * STDEV([Parameter])))) AS DECIMAL(10, 2)) >
                    CAST(CONVERT(FLOAT, ((AVG([Parameter]) - [LSL]) / (3 * STDEV([Parameter])))) AS DECIMAL(10, 2))) AND [LSL] = 0)
                THEN CAST(CONVERT(FLOAT, (([USL] - AVG([Parameter])) / (3 * STDEV([Parameter])))) AS DECIMAL(10, 2))
            WHEN CAST(CONVERT(FLOAT, (([USL] - AVG([Parameter])) / (3 * STDEV([Parameter])))) AS DECIMAL(10, 2)) >
                 CAST(CONVERT(FLOAT, ((AVG([Parameter]) - [LSL]) / (3 * STDEV([Parameter])))) AS DECIMAL(10, 2))
                THEN CAST(CONVERT(FLOAT, ((AVG([Parameter]) - [LSL]) / (3 * STDEV([Parameter])))) AS DECIMAL(10, 2))
            WHEN CAST(CONVERT(FLOAT, (([USL] - AVG([Parameter])) / (3 * STDEV([Parameter])))) AS DECIMAL(10, 2)) =
                 CAST(CONVERT(FLOAT, ((AVG([Parameter]) - [LSL]) / (3 * STDEV([Parameter])))) AS DECIMAL(10, 2))
                THEN CAST(CONVERT(FLOAT, (([USL] - AVG([Parameter])) / (3 * STDEV([Parameter])))) AS DECIMAL(10, 2))
           END AS [CPK],
           [model] AS [Model],
           [MC] AS [Machine],
           [Line] AS Line
    FROM XT
    GROUP BY [Date], [model], [LSL], [CL], [USL], [MC], [Line]
	HAVING  STDEV([Parameter]) != 0 and STDEV([Parameter]) is not null

)
-- คำสั่ง SQL ส่วนอื่น ๆ
SELECT [Date], [%Yield], [Cycle_time (sec)], [Down time (min)], [AVG], [STD], [CPK], [Model], [Machine], [Line],@Parameter as Parameter
FROM XO;
        """
datasets = pd.read_sql(query, cnxn)
print(datasets.head(100000))

C:\Users\IT\AppData\Local\Temp\ipykernel_4964\1646507424.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  max_date_df = pd.read_sql(max_date_query, cnxn)
C:\Users\IT\AppData\Local\Temp\ipykernel_4964\1646507424.py:112: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  datasets = pd.read_sql(query, cnxn)


Maximum Date + 1 day: 2023-08-29
           Date  %Yield  Cycle_time (sec)  Down time (min)    AVG     STD  \
0    2023-08-09  100.00              2.18              0.0  4.447  0.0044   
1    2023-08-09  100.00              1.02              0.0  3.188  0.0075   
2    2023-08-10   99.93              0.38              0.0  1.346  0.0101   
3    2023-08-11   99.76              0.54              0.0  0.838  0.0093   
4    2023-08-11   99.37              0.47              0.0  4.456  0.0081   
..          ...     ...               ...              ...    ...     ...   
128  2023-08-17   99.73              0.74              0.0  1.333  0.0109   
129  2023-08-17   99.93              0.76              0.0  1.344  0.0098   
130  2023-08-19  100.00              0.74              0.0  1.331  0.0106   
131  2023-08-19  100.00              0.61              0.0  3.194  0.0073   
132  2023-08-19  100.00              0.74              0.0  3.191  0.0073   

      CPK   Model     Machine  Line  Param

In [3]:
# Your existing code here...

# Execute the SQL query and store the results in the 'datasets' DataFrame
datasets = pd.read_sql(query, cnxn)

# Your existing code here...

# Establish a new connection for inserting the data into the target table
insert_server = '192.168.101.219'
insert_database = 'DataforAnalysis'
insert_username = 'DATALYZER'
insert_password = 'NMB54321'
insert_cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+insert_server+';DATABASE='+insert_database+';UID='+insert_username+';PWD='+ insert_password)
insert_cursor = insert_cnxn.cursor()

# Iterate through the rows of the 'datasets' DataFrame and insert data into the target table
for index, row in datasets.iterrows():
    insert_query = f"""
        INSERT INTO [DataforAnalysis].[dbo].[Store_SPC] ([Date], [Model], [Line], [Machine], [%Yield], [Cycle_time (sec)],
        [Down time (min)], [AVG], [STD], [CPK], [Parameter])
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    values = (row['Date'], row['Model'], row['Line'], row['Machine'], row['%Yield'], row['Cycle_time (sec)'],
              row['Down time (min)'], row['AVG'], row['STD'], row['CPK'], row['Parameter'])
    insert_cursor.execute(insert_query, values)

# Commit the changes and close the connection
insert_cursor.commit()
insert_cursor.close()
insert_cnxn.close()


C:\Users\IT\AppData\Local\Temp\ipykernel_4964\3032691257.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  datasets = pd.read_sql(query, cnxn)


In [4]:
import pyodbc
import pandas as pd
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = '192.168.101.219' 
database = 'DataforAnalysis' 
username = 'DATALYZER' 
password = 'NMB54321'  
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# select 26 rows from SQL table to insert in dataframe.
# Query to get the maximum date
max_date_query = """
    SELECT DATEADD(day, 1, MAX([Date])) as MaxDate
    FROM [DataforAnalysis].[dbo].[SPC_STD_MC];
"""
max_date_df = pd.read_sql(max_date_query, cnxn)

# Printing the maximum date with one day added
max_date_plus_1 = max_date_df['MaxDate'][0]
print(f"Maximum Date + 1 day: {max_date_plus_1}")

query = f"""
  DECLARE @DynamicSQL AS NVARCHAR(max),
 @Start NVARCHAR(30) = '{max_date_plus_1}',
 @End NVARCHAR(30) = '{finishDate}',
 @Parameter NVARCHAR(30) = 'Set_Dim_B';

    WITH BarcodeThan100 as(
      select [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Date] as [Date]
             ,count([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Barcode]) as  [countBarcode]
             from [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester]
                INNER JOIN [TransportData].[dbo].[Master_matchings]
                ON [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].Model = [TransportData].[dbo].[Master_matchings].Model
             where [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Date] BETWEEN  @Start AND  @End
             and [TransportData].[dbo].[Master_matchings].[createdAt] = (select max([TransportData].[dbo].[Master_matchings].[createdAt]) from [TransportData].[dbo].[Master_matchings])
       group by [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Date]
       )
    ,XT (x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11) as
        (select [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Date] as [Date]
        ,case when datediff(SECOND,0,cast((lag([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) over (order by [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] desc) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) as time)) < 120
        then datediff(SECOND,0,cast((lag([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) over (order by [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] desc) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) as time))
        when datediff(SECOND,0,cast((lag([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) over (order by [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] desc) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) as time)) >= 120 then 0 end as [C/T in sec]
        ,case when datediff(SECOND,0,cast((lag([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) over (order by [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] desc) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) as time)) >= 120
        then datediff(SECOND,0,cast((lag([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) over (order by [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] desc) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) as time))
        when datediff(SECOND,0,cast((lag([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) over (order by [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] desc) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) as time)) < 120 then 0 end as [D/T]
        ,[DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].Set_Dim_B as [Parameter]
        ,[DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Model] as [model]
        ,[TransportData].[dbo].[Master_matchings].LSL as [LSL]
        ,[TransportData].[dbo].[Master_matchings].CL as [CL]
        ,[TransportData].[dbo].[Master_matchings].USL as [USL]
        ,'Average' as [AVG]
        ,[DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Barcode]
		,Line
        from BarcodeThan100
                                left join [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester]
                                on BarcodeThan100.Date = [Dynamic_Parallelism_Tester].Date
        INNER JOIN [TransportData].[dbo].[Master_matchings]
        ON [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].Model = [TransportData].[dbo].[Master_matchings].Model
         where [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Date] BETWEEN  @Start AND  @End
        and [TransportData].[dbo].[Master_matchings].[Parameter] = @Parameter
        and [TransportData].[dbo].[Master_matchings].[createdAt] = (select max([TransportData].[dbo].[Master_matchings].[createdAt]) from [TransportData].[dbo].[Master_matchings])
        and countBarcode >100
        and Operator = 'Operator'
        ),
     XO (x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13) as (
    select 
        x1 as [Date],
        ISNULL(cast(convert(float,count(case when x4 >= x6 and x4 <= x8 then 1 else null end))/
        convert(float,count(case when x4 >= x6 and x4 <= x8 then 1 else null end)
        + count(case when x4 < x6 or x4 > x8 then 1 else null end)) * 100 AS DECIMAL(10, 2)), 0) as [%Yield], -- Replace NULL with 0
        ISNULL(cast(convert(float,sum(x2))/count(x2) as DECIMAL(10, 2)), 0) as [Cycle_time (sec)], -- Replace NULL with 0
        ISNULL(cast(convert(float,sum(x3))/60 as decimal(10,0)), 0) as [Down time (min)], -- Replace NULL with 0
        ISNULL(cast(AVG(x4) as decimal(10,3)), 0) as [AVG], -- Replace NULL with 0
        ISNULL(cast(stdev(x4) as decimal(10,4)), 0) as [STD], -- Replace NULL with 0
        case when (cast(convert(float,((x8-AVG(x4))/(3*STDEV(x4)))) as decimal(10,2)) < cast(convert(float,((AVG(x4)-x6)/(3*STDEV(x4)))) as decimal(10,2))) or
        ((cast(convert(float,((x8-AVG(x4))/(3*STDEV(x4)))) as decimal(10,2)) > cast(convert(float,((AVG(x4)-x6)/(3*STDEV(x4)))) as decimal(10,2))) and x6 = 0)
        then ISNULL(cast(convert(float,((x8-AVG(x4))/(3*STDEV(x4)))) as decimal(10,2)), 0) when cast(convert(float,((x8-AVG(x4))/(3*STDEV(x4)))) as decimal(10,2))
        > cast(convert(float,((AVG(x4)-x6)/(3*STDEV(x4)))) as decimal(10,2))
        then ISNULL(cast(convert(float,((AVG(x4)-x6)/(3*STDEV(x4)))) as decimal(10,2)), 0) when cast(convert(float,((x8-AVG(x4))/(3*STDEV(x4)))) as decimal(10,2))
        = cast(convert(float,((AVG(x4)-x6)/(3*STDEV(x4)))) as decimal(10,2))
        then ISNULL(cast(convert(float,((x8-AVG(x4))/(3*STDEV(x4)))) as decimal(10,2)), 0) end as [CPK],
        x5 as [Model],
        x6 as [LSL],
        x7 as [CL],
        x8 as [USL],
        x9 as [Machine],
        x11 as Line
    from XT
    group by x1,x5,x6,x7,x8,x9,x11
    having STDEV(x4) != 0 and STDEV(x4) is not null
)
      select 
      XO.x1 as [Date]
	  ,XO.x8 as [Model]
      ,XO.x13 as Line
      ,XO.x12 as [Machine]
      ,XO.x2 as [%Yield]
      ,XO.x3 as [Cycle_time (sec)]
      ,XO.x4 as [Down time (min)]
      ,XO.x5 as [AVG]
      ,XO.x6 as [STD]
      ,XO.x7 as [CPK]
    
      ,x11 as [USL]
      ,x10 as [CL]
      ,x9 as [LSL]
	  ,@Parameter as Parameter
      from XO
        """
dataSTD = pd.read_sql(query, cnxn)
print(dataSTD.head(100000))

C:\Users\IT\AppData\Local\Temp\ipykernel_4964\1707701269.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  max_date_df = pd.read_sql(max_date_query, cnxn)
C:\Users\IT\AppData\Local\Temp\ipykernel_4964\1707701269.py:112: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataSTD = pd.read_sql(query, cnxn)


Maximum Date + 1 day: 2023-08-29
          Date   Model  Line  Machine  %Yield  Cycle_time (sec)  \
0   2023-08-09  V15C4D  3-22  Average  100.00              0.79   
1   2023-08-11  M11P5D  2-14  Average   99.65              0.51   
2   2023-08-17  LONGSP   3-6  Average   97.87              0.47   
3   2023-08-25  V11 4D  3-22  Average  100.00              1.54   
4   2023-08-25  V15C1D   2-9  Average  100.00              2.94   
..         ...     ...   ...      ...     ...               ...   
78  2023-08-17  V11 4D  3-22  Average  100.00              0.67   
79  2023-08-17  V15C2D  3-29  Average   99.95              0.62   
80  2023-08-17  V15C4D  3-22  Average   99.93              0.91   
81  2023-08-19  V11 2D  3-29  Average  100.00              0.62   
82  2023-08-28  EVANBP  2-10  Average  100.00             15.40   

    Down time (min)    AVG     STD   CPK     USL      CL     LSL  Parameter  
0               0.0  1.761  0.0082  1.49  1.8009  1.7628  1.7247  Set_Dim_B  
1     

In [5]:
# Your existing code here...

# Execute the SQL query and store the results in the 'datasets' DataFrame
datasets = pd.read_sql(query, cnxn)

# Your existing code here...

# Establish a new connection for inserting the data into the target table
insert_server = '192.168.101.219'
insert_database = 'DataforAnalysis'
insert_username = 'DATALYZER'
insert_password = 'NMB54321'
insert_cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+insert_server+';DATABASE='+insert_database+';UID='+insert_username+';PWD='+ insert_password)
insert_cursor = insert_cnxn.cursor()

# Iterate through the rows of the 'datasets' DataFrame and insert data into the target table
for index, row in dataSTD.iterrows():
    insert_query = f"""
        insert into [DataforAnalysis].[dbo].[SPC_STD_MC] ([Date], [Model], [Line], [Machine], [%Yield], [Cycle_time (sec)],
        [Down time (min)], [AVG], [STD], [CPK],[LSL],[CL],[USL],[Parameter])
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ? , ? )
    """
    values = (row['Date'], row['Model'], row['Line'], row['Machine'], row['%Yield'], row['Cycle_time (sec)'],
              row['Down time (min)'], row['AVG'], row['STD'], row['CPK'],row['LSL'],row['CL'],row['USL'], row['Parameter'])
    insert_cursor.execute(insert_query, values)

# Commit the changes and close the connection
insert_cursor.commit()
insert_cursor.close()
insert_cnxn.close()

C:\Users\IT\AppData\Local\Temp\ipykernel_4964\4262995815.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  datasets = pd.read_sql(query, cnxn)
